<a href="https://colab.research.google.com/github/zzbi007zz/AI-Screenshot/blob/main/Vintern_1B_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U -q transformers==4.44.2 bitsandbytes
!pip install -U -q huggingface_hub
!pip install -q flask flask-cors pyngrok flash_attn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/

In [2]:
import numpy as np
import torch
import torchvision.transforms as T
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer
import requests

In [3]:
# Thư viện xử lý ảnh đầu vào (từ source gốc HF)
IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(requests.get(image_file, stream=True).raw).convert('RGB')#Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

In [ ]:
model_name = "5CD-AI/Vintern-1B-v2"
model = AutoModel.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
).eval().cuda()

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False)
generation_config = dict(max_new_tokens= 512, do_sample=False, num_beams = 3, repetition_penalty=3.5)


In [6]:
test_image = 'https://media-cdn-v2.laodong.vn/Storage/NewsPortal/2022/12/7/1124909/Karaoke-2.jpg'

pixel_values = load_image(test_image, max_num=6).to(torch.bfloat16).cuda()
generation_config = dict(max_new_tokens= 1024, do_sample=False, num_beams = 3, repetition_penalty=2.5)

question = '<image>\nTrích xuất thông tin chính trong ảnh và trả về dạng json.'

response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')



Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


User: <image>
Trích xuất thông tin chính trong ảnh và trả về dạng markdown.
Assistant: HÓA ĐƠN TẠM TÍNH
ĐC: Số 51, Đường Nguyễn Hoàng - P. Mỹ Đình 2 - Q. Nam Từ Liêm - TP. Hà Nội
ĐT: 024 3932 5999
Hotline: 0969 61 61 99 - 0962 61 61 99
Ngày: 04.12.2022
Giờ vào: 19.22
Bàn: VIP 222
Số Bill: 21.18
Giờ ra: 21.18
Thu ngân: THU NGÂN 1
STT | Tên món | SL | Đơn giá | Giảm giá | T.Tiền
1 | Giờ VIP222 | 1h54' | 500 000 | 0% | 950 000
2 | Suối | 1 | 12 000 | 0% | 36 000
3 | Hoa quả thập cẩm | 1 | 140 000 | 0% | 140 000
4 | Hoa quả Bưởi | 2 | 220 000 | 0% | 440 000
5 | Hoa Quả Roi | 1 | 100 000 | 0% | 100 000
6 | Ken ngoại | 14 | 60 000 | 0% | 840 000
Thành tiền: 2 506 000
Tiền Thanh Toán: 2 506 000


In [13]:

# Setup Ngrok Token
from google.colab import userdata
from flask import Flask, jsonify, request
from flask_cors import CORS
from pyngrok import ngrok

authtoken = userdata.get("ngrok_token")
ngrok.set_auth_token(authtoken)

In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import torch
from PIL import Image
import requests

app = Flask(__name__)
CORS(app)

prompt = '''<image>\nNhận diện hoá đơn trong ảnh. Chỉ trả về phần liệt kê các mặt hàng hàng dưới dạng CSV'''

@app.route('/ocr', methods=['POST'])
def index():
    data = request.json
    image_url = data.get('image_url', None)
    response_message = ocr_by_llm(image_url, prompt)
    return jsonify({"response_message": response_message})

def ocr_by_llm(image_url, prompt):
    pixel_values = load_image(image_url, max_num=6).to(torch.bfloat16).cuda()
    response_message = model.chat(tokenizer, pixel_values, prompt, generation_config)
    del pixel_values
    print(response_message)
    return response_message

if __name__ == '__main__':
    ngrok_url = ngrok.connect(5555)
    print('Public URL:', ngrok_url)
    app.run(port=5555)

Public URL: NgrokTunnel: "https://8172-35-197-4-76.ngrok-free.app" -> "http://localhost:5555"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5555
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [13/Feb/2025 01:51:49] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/Feb/2025 01:51:50] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/Feb/2025 01:51:50] "GET /favicon.ico HTTP/1.1" 404 -
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [13/Feb/2025 02:28:56] "POST /ocr HTTP/1.1" 200 -


| TT | Tên món | SL | Đơn giá | Thành tiền |
|---|---|---|---|---|
| 1 | Giờ hát | 4h11' | 200 000 | 836 000 |
| 2 | Oanh | 1,50 | 200 000 | 300 000 |
| 3 | Xuân | 2,83 | 200 000 | 566 000 |
| 4 | Linh | 4,10 | 200 000 | 820 000 |
| 5 | Hoa quả đĩa | 1,00 | 150 000 | 150 000 |
| 6 | Thuốc Thăng Long | 1,00 | 20 000 | 20 000 |
| 7 | Minh | 2,33 | 200 000 | 466 000 |
| 8 | Bia | 23,00 | 20 000 | 460 000 |
| 9 | Trà xanh Ô độ | 1,00 | 23 000 | 23 000 |
| Thành tiền: | 3 641 000 | 3 641 000 |
| Tiền phí dịch vụ | 5% | 182 050 |
| Tổng cộng: | 3 823 050 | 3 823 050 |
| Tiền Khách Dưa: | 0 | 0 |
| Tiền Trà Khách: | 0 | 0 |
| Tiền mặt (VND): | 3 823 050 | 3 823 050 |


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [13/Feb/2025 02:31:53] "POST /ocr HTTP/1.1" 200 -


[['Loại', 'Thứ hai', 'Ngày'], ['22-T01K5', 'Thứ hai', '31-01-2022']]


...

['Loại TGB3', 'Một', 'Bốn', 'Năm', 'Tám', 'Năm', 'Một']


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [13/Feb/2025 02:34:15] "POST /ocr HTTP/1.1" 200 -


['Loại TGB3', 'Một', 'Bốn', 'Năm', 'Tám', 'Năm', 'Một']


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
INFO:werkzeug:127.0.0.1 - - [13/Feb/2025 02:34:34] "POST /ocr HTTP/1.1" 200 -


['Loại TGB3', 'Một', 'Bốn', 'Năm', 'Tám', 'Năm', 'Một']
